# Run a Spark Job on AzureML

This notebook provides an example of how to define and run a job on AzureML using Spark. This notebook is the _control plane_, meaning it creates a connection to the AzureML workspace, defines the job, and submits the job.

**This Jupyter notebook should be run from within a compute instance on AzureML, in a Python kernel, specifically `Python 3.10 - SDK v2 (Python 3.10.11)`**. 

As you can see from the files contained in this `job` subdirectory, there are several files:

- A parametrized Python script with pyspark code that is submitted to a Spark cluster



## Create a client connection to the AzureML workspace

The following cell creates a connection object called `azureml_client` which has a connection to the AzureML workspace.

In [17]:
from azure.ai.ml import MLClient, spark, Input, Output
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import UserIdentityConfiguration

azureml_client = MLClient.from_config(
    DefaultAzureCredential(),
)


Found the config file in: /config.json


## Define the Job

The following cell defines the job. It is an object of [Spark Class](https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.spark?view=azure-python) that contains the required information to run a job:

- The cluster size
- The script to run
- The parameters for the script

In the example below, we are using the `pyspark-script-job.py` script which is parametrized. As you can see, the parameters are the following:

- `input_object_store_base_url` (**don't forget the trailing slashes /**): 
    - Here you will use a base URL of the `s3://<BUCKETNAME>/` form for Sagemaker, 
    - or `wasbs://<CONTAINER-NAME>@<STORAGE-ACCOUNT>.blob.core.windows.net/` 
    - or `azureml://datastores/workspaceblobstore/paths/` for AzureML. **Don't forget the trailing slash /.**
- `input_path`: The path to read from
- `output_object_store_base_url`: 
- `output_path`: The path to write to
- `subreddits`: a comma separated string of subreddit names

The PySpark script accepts the object store location for the raw data, in this case a single month. Then the job filters the original data and writes the filtered data out. This is designed to be used for either submissions or comments, not both.

For more information about the parameters used in the job definition, [read the documentation](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-submit-spark-jobs?view=azureml-api-2&tabs=sdk#submit-a-standalone-spark-job).



In [19]:
workspace_default_storage_account = "projectgstoragedfb938a3e"
workspace_default_container = "azureml-blobstore-becc8696-e562-432e-af12-8a5e3e1f9b0f"
workspace_wasbs_base_url = f"wasbs://{workspace_default_container}@{workspace_default_storage_account}.blob.core.windows.net/"
workspace_wasbs_base_url = "wasbs://azureml-blobstore-becc8696-e562-432e-af12-8a5e3e1f9b0f@projectgstoragedfb938a3e.blob.core.windows.net/"

job1_comments = spark(
    description="cancer-comments",
    code="./",
    entry={"file": "pyspark-reddit-filter-script-job.py"},
    driver_cores=1,
    driver_memory="7g",
    executor_cores=4,
    executor_memory="7g",
    executor_instances=6,
    resources={
        "instance_type": "Standard_E4S_V3",
        "runtime_version": "3.4",
    },
    inputs={       
        "input_object_store_base_url": "wasbs://reddit-project@dsan6000fall2024.blob.core.windows.net/",
        "input_path": "202306-202407/comments/",
        "output_object_store_base_url": "wasbs://azureml-blobstore-becc8696-e562-432e-af12-8a5e3e1f9b0f@projectgstoragedfb938a3e.blob.core.windows.net/",
        "output_path": "cancer/comments/",
        "subreddits": "CrohnsDisease,thyroidcancer,AskDocs,UlcerativeColitis,Autoimmune,BladderCancer,breastcancer,CancerFamilySupport,doihavebreastcancer,WomensHealth,ProstateCancer,cll,Microbiome,predental,endometrialcancer,cancer,Hashimotos,coloncancer,PreCervicalCancer,lymphoma,Lymphedema,CancerCaregivers,braincancer,lynchsyndrome,nursing,testicularcancer,leukemia,publichealth,Health,Fuckcancer,HealthInsurance,BRCA,Cancersurvivors,pancreaticcancer,skincancer,stomachcancer"
    },
    identity=UserIdentityConfiguration(),
    args="--input_object_store_base_url ${{inputs.input_object_store_base_url}} --input_path ${{inputs.input_path}} --output_object_store_base_url ${{inputs.output_object_store_base_url}} --output_path ${{inputs.output_path}} --subreddits ${{inputs.subreddits}}"
)

job2_submissions = spark(
    description="cancer-submissions",
    code="./",
    entry={"file": "pyspark-reddit-filter-script-job.py"},
    driver_cores=1,
    driver_memory="7g",
    executor_cores=4,
    executor_memory="7g",
    executor_instances=6,
    resources={
        "instance_type": "Standard_E4S_V3",
        "runtime_version": "3.4",
    },
    inputs={       
        "input_object_store_base_url": "wasbs://reddit-project@dsan6000fall2024.blob.core.windows.net/",
        "input_path": "202306-202407/submissions/",
        "output_object_store_base_url": "wasbs://azureml-blobstore-becc8696-e562-432e-af12-8a5e3e1f9b0f@projectgstoragedfb938a3e.blob.core.windows.net/",
        "output_path": "cancer/submissions/",
        "subreddits": "CrohnsDisease,thyroidcancer,AskDocs,UlcerativeColitis,Autoimmune,BladderCancer,breastcancer,CancerFamilySupport,doihavebreastcancer,WomensHealth,ProstateCancer,cll,Microbiome,predental,endometrialcancer,cancer,Hashimotos,coloncancer,PreCervicalCancer,lymphoma,Lymphedema,CancerCaregivers,braincancer,lynchsyndrome,nursing,testicularcancer,leukemia,publichealth,Health,Fuckcancer,HealthInsurance,BRCA,Cancersurvivors,pancreaticcancer,skincancer,stomachcancer"
    },
    identity=UserIdentityConfiguration(),
    args="--input_object_store_base_url ${{inputs.input_object_store_base_url}} --input_path ${{inputs.input_path}} --output_object_store_base_url ${{inputs.output_object_store_base_url}} --output_path ${{inputs.output_path}} --subreddits ${{inputs.subreddits}}"
)



## Submit the job

The following cell takes the job you defined above and submits it. If you are submitting multiple jobs, you may want to create separate job definition objects for clarity. You can submit more than one job, just remember that each job will spin up a Spark cluster.

In [20]:
job1_object = azureml_client.jobs.create_or_update(job1_comments)

Uploading simple-spark-job-with-parameters-copy (0.03 MBs): 100%|██████████| 28961/28961 [00:00<00:00, 316941.30it/s]




In [21]:
job2_object = azureml_client.jobs.create_or_update(job2_submissions)

## Get the Job Studio URL

Once you submit the job, you can navigate to it in the AzureML Studio and monitor it's progress. There are ways to do it through the SDK but for now just use the Studio. These are unattended jobs, which means you can shut down this notebook and the Compute Instance, but the job will go through it's lifecycle:

- Job is submitted
- Job is queued
- Job is run
- Job completes (assuming no errors)

**Each job's Studio URL will be different.**

In [22]:
job1_url = job1_object.studio_url
print(job1_url)

https://ml.azure.com/runs/sleepy_town_yj06hs7n2m?wsid=/subscriptions/21ff0fc0-dd2c-450d-93b7-96eeb3699b22/resourcegroups/project-group-35/workspaces/project-group-35&tid=fd571193-38cb-437b-bb55-60f28d67b643


In [23]:
job2_url = job1_object.studio_url
print(job2_url)

https://ml.azure.com/runs/sleepy_town_yj06hs7n2m?wsid=/subscriptions/21ff0fc0-dd2c-450d-93b7-96eeb3699b22/resourcegroups/project-group-35/workspaces/project-group-35&tid=fd571193-38cb-437b-bb55-60f28d67b643
